# socket

## 服务端

In [ ]:
import socket
import time
import threading

local_addr = ('0.0.0.0', 10015)
buff_len = 1024


def conn_proc(sock, addr):

    print('accept new connection from %s:%s...' % addr)
    # 发送数据给客户端
    sock.send('welcome'.encode('utf-8'))

    while True:

        print('wating next message...')
        # 阻塞接收客户端数据
        date = sock.recv(buff_len)
        if not date or date.decode('utf-8') == 'exit':
            break
        print(date.decode('utf-8'))
        sock.send(('server ' + str(time.time())).encode('utf-8'))

    sock.close()
    print('close connection %s:%s' % addr)


if __name__ == '__main__':

    # 创建一个socket
    tcp = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 绑定监听的地址和端口
    tcp.bind(local_addr)
    # 开始监听端口，等待连接的最大数量为10
    tcp.listen(10)

    while True:
        # 阻塞等待客户端过来的tcp连接
        sock, addr = tcp.accept()
        print(type(sock), type(addr))
        # 创建一个新线程来处理TCP连接
        threading.Thread(target=conn_proc, args=(sock, addr)).start()

    # 关闭socket
    tcp.close()

## 客户端

In [ ]:
#!/usr/bin/env python
#coding=utf-8

import time
import socket

server_addr = ('127.0.0.1', 12345)
buff_len = 1024

if __name__ == '__main__':

    # 创建一个socket
    tcp = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    # 建立连接
    tcp.connect(server_addr)

    # 发送数据给服务器
    tcp.send('hello server.'.encode('utf-8'))
    # 阻塞接收服务器数据
    print(tcp.recv(buff_len).decode('utf-8'))

    while True:
        text = input(">>")
        tcp.send(text.encode('utf-8'))
        date = tcp.recv(buff_len)
        if not date or date.decode('utf-8') == 'exit':
            break
        print(date.decode('utf-8'))

    # 关闭socket
    tcp.close()

# socketserver

## 服务端

In [ ]:
import socketserver


class MyHandler(socketserver.BaseRequestHandler):
    def handle(self):
        # 通信循环
        while True:
            # print(self.client_address)
            # print(self.request) #self.request=conn
            try:
                data = self.request.recv(1024)
                print('receive {}'.format(data.decode('utf-8')))
                if len(data.strip()) == 0: break
                self.request.send(data.upper())
            except ConnectionResetError:
                break


if __name__ == '__main__':
    s = socketserver.ThreadingTCPServer(('127.0.0.1', 12345), MyHandler, bind_and_activate=True)
    s.serve_forever()  # 代表连接循环


In [ ]:
import socketserver
from datetime import datetime
import threading
import sys


class MyHandler(socketserver.BaseRequestHandler):

    def task_listen(self):
        while True:
            try:
                data = self.request.recv(4096)
            except OSError as e:
                break
            if data:
                try:
                    sys.stdout.write(data.decode())
                except UnicodeDecodeError:
                    sys.stdout.write(data)
                sys.stdout.write('\n')

    def task_send(self):
        while True:
            try:
                data = sys.stdin.readline()
                print('输入：', data)
                if data.strip():
                    self.request.send(data.encode())
                else:
                    break
            except ConnectionResetError:
                break

    def handle(self):
        print('客户端已连接', self.client_address)
        th1 = threading.Thread(target=self.task_listen, daemon=True)
        th2 = threading.Thread(target=self.task_send, daemon=True)

        th1.start()
        th2.start()
        th2.join()
        print('程序结束')
        self.server.shutdown()


if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument('-p', '--port', type=int)
    args = parser.parse_args()
    if args.port:
        s = socketserver.ThreadingTCPServer(('0.0.0.0', args.port), MyHandler, bind_and_activate=True)
        s.serve_forever()  # 代表连接循环
    else:
        print(parser.format_help())